In [1]:
from openai import OpenAI
from api_keys import openai_key
client = OpenAI(api_key=openai_key)

def get_completion(prompt,model="gpt-3.5-turbo",response_format="json_object"):
  response = client.chat.completions.create(
    model=model,
    # response_format = {"type":response_format},
    messages=[
      {"role": "user", "content":prompt}
    ]
  )
  return response.choices[0].message.content, response

In [2]:
r = get_completion('Provide a prompt for gpt 3.5 turbo to read a text file which contains a table with columns separated by spaces in a line, and the multiple lines contain values for the same row. every new row starts with a non empty space in the starting of the line',model='gpt-4')
r[0]

('"Given a text file that contains a table where columns are separated by spaces and each line contains values for the same row. A new row begins with a non-empty space at the start of the line. Please read this text file, interpret the data, and organize it into a tabular format for further analysis."',
 ChatCompletion(id='chatcmpl-9HUgi5TOu2sRlAu2DMDn2W1xafH6V', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='"Given a text file that contains a table where columns are separated by spaces and each line contains values for the same row. A new row begins with a non-empty space at the start of the line. Please read this text file, interpret the data, and organize it into a tabular format for further analysis."', role='assistant', function_call=None, tool_calls=None))], created=1713956568, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=63, prompt_tokens=62, total_tokens=125)

In [5]:
from azure_ocr import read_ocr

fil = 'pdfs-samples/1. Khushhali Bank 1 _KMBL_PKR.pdf'
f = fil[13:-4]

text, pages = read_ocr(fil,f)

In [13]:
p = """
Given a text file of bank statement with transactions
in the form of a table where columns 
are separated by spaces and each line contains values 
for the same row. A new row begins with a non-empty 
space at the start of the line. Please read this text 
file, interpret the data, and organize it into a tabular 
format for further analysis. output the data in a csv 
format with the columns:
Date,Description,Withdrawal,Deposit,Balance
####
Input: 
"{text}
"
Output :

"""
r = get_completion(p.format(text=pages[1]))
with open('temp.csv','w') as ff:
    ff.write(r[0])
r[0]


'Date,Description,Withdrawal,Deposit,Balance\n15 AUG 23,WHT on Profit,3821.92,573457.27,\n15 AUG 23,WHT on Profit,5732.88,567724.39,\n5 AUG 23,WHT on Profit,5732.88,561991.51,\n5 AUG 23,WHT on Profit,955.48,561036.03,\n15 AUG 23,WHT on Profit,3821.92,557214.11,\n15 AUG 23,WHT on Profit,1910.96,555 303.15,\n31 AUG 23,Profit - Capitalise,,4891.47,560194.62,\n31 AUG 23,Debit Arrangement,733.72,559460.90,\n14 SEP 23,Credit Arrangement,,25479.45,584940.35,\n14 SEP 23,Credit Arrangement,,38219.18,623159.53,\nSEP 23,Credit Arrangement,,25479.45,648638.98,\n14 SEP 23,Credit Arrangement,,6369.86,655008.84,\n14 SEP 23,Credit Arrangement,,38219.18,693228.02,\n14 SEP 23,Credit Arrangement,,6369.86,699597.88,\n14 SEP 23,Credit Arrangement rement,,12739.73,712337.61,\n14 SEP 23,WHT on Profit,955.48,711382.13,\n14 SEP 23,WHT on Profit,3821.92,707560.21,\n14 SEP 23,WHT on Profit,1910.96,705649.25,\n14 SEP 23,WHT on Profit,3821.92,701827.33,\n14 SEP 23,WHT on Profit,5732.88,696094.45,\n14 SEP 23,WHT on

In [8]:
import pandas as pd
df = pd.read_csv('temp.csv')
df

,Date,Description,Withdrawal,Deposit,Balance
0,14 JUL 23,Credit Arrangement,NaN,6164.38,303363.55
1,14 JUL 23,Credit Arrangement,NaN,6164.38,309527.93
2,14 JUL 23,Credit Arrangement,NaN,24657.53,334185.46
3,14 JUL 23,Credit Arrangement,NaN,36986.30,371171.76
4,14 JUL 23,Credit Arrangement,NaN,12328.77,383500.53
5,14 JUL 23,Credit Arrangement,NaN,24657.53,408158.06
6,14 JUL 23,Credit Arrangement,NaN,36986.30,445144.36
7,14 JUL 23,WHT on Profit,5547.95,NaN,439596.41
8,14 JUL 23,WHT on Profit,3698.63,NaN,435897.78
9,14 JUL 23,WHT on Profit,1849.32,NaN,434048.46


In [8]:
r = get_completion("""
my task is to extract transactions from a bank statement, i am using gpt-3.5-turbo
to extract the transactions. any reason why gpt is missing out transactions when i 
                   use the following prompt. give me reasons and suggest improvements.

                   "You have been tasked as an AI data analyst to meticulously process the information 
presented in a given bank credit statement text. Your role is to accurately interpret, 
extract, and present each transaction in this statement in CSV Format with the following 
column labels: "Transaction Date", "Description", "Debit/Withdrawal", "Credit/Deposit", "Balance". 

It's crucial you don't miss any transaction and represent each transaction exactly as they 
occur in the statement text without alterations, summarizations or omissions. Dates should be 
accurately represented in the "DD-MM-YYYY" format. Transaction descriptions can spread across 
several lines, but every description will harbor a numerical value in either the "Debit/Withdrawal" 
or the "Credit/Deposit" column. 

For "Debit/Withdrawal", this will be denoted as the third last numerical value and could also be identified 
with a "CR" description or a negative numerical value. "Credit/Deposit" will be represented as the 
second last numerical value and could be recognized with a "DR" description. The "Balance" figure 
will be the last numerical value contained in each transaction line and each value will differ 
per transaction. 

Be aware that each row can only shelter either a "Debit/Withdrawal" or "Credit/Deposit" value but not both 
at once. Whenever a transaction doesn't comprise either a "Debit/Withdrawal" or "Credit/Deposit", mark it as 0.0. 

The final output should be a comprehensive list of all transactions which occurred in the document, 
represented solely in JSON format with fields including: "user_bank", "opening_balance_date", "opening_balance", 
"closing_balance", "transactions". 
Note that the output should only contain numerical values and decimals where applicable, without the 
inclusion of special characters. Here is your text to extract transactions from:
{text}."
""")
r

'The prompt might have a few issues that could contribute to GPT-3.5-turbo missing transactions in the bank statement:\n\n1. Complexity: This prompt is quite detailed and long, potentially confusing the model. GPT-3.5-turbo might have trouble picking up the essential information to execute the task correctly.\n\n2. Specific Terminology: The use of specific terms like "third last numerical value" or "CR" description, or the way that Debit/Withdrawal and Credit/Deposit transactions are explained, might not align with the format presented in the text. This mismatch might result in the model missing transactions.\n\n3. Transaction Description: The prompt mentions that transaction descriptions can be spread across several lines. Depending on how text is formatted, this might cause the model to skip transactions.\n\nTo improve the prompt, I would suggest the following changes:\n\n1. Simplify the prompt: A simpler, more straightforward prompt might work better. \n\n2. Generic Terminology: Ins

In [ ]:
get_completion("if I have to apply for daad what things do I need", response_format="text")

In [ ]:
def read_pdf(file_path):
  with pdfplumber.open(file_path) as pdf:
    text = ""
    pages = []
    for page in pdf.pages:
      p = page.extract_text()
      text += p
      pages.append(p)
  return text, pages

# Example usage
file_path = "Allied Bank.pdf"
text, pages = read_pdf(file_path)
print(text)


ABC (Private) Limited PK0010529Branch Name:HALL ROAD LAHORE
Statement Period:01 JUL 2021 TO 30 JUN 2022
Account Number:0000000000000000
Account Status:Regular
Pakistan Rupee
Current Accounts
BALANCE AT PERIOD START : 914,280.76
------------------------------------------------------------------------------------------------------------
DATE PARTICULARS VALUE DATE DEBITS CREDITS BALANCE
------------------------------------------------------------------------------------------------------------
05 JUL 21 Cash Withdrawal 05 JUL 21 26,610.00 887,670.76
51451190
05 JUL 21 Account Transfer 05 JUL 21 19,100.00 868,570.76
51451189
06 JUL 21 Online Cash Deposit CR. 06 JUL 21 25,000.00 893,570.76
PK0010539
06 JUL 21 Transfer 06 JUL 21 130,480.00 1,024,050.76
67279356
06 JUL 21 Cash Withdrawal 06 JUL 21 80,950.00 943,100.76
51451193
07 JUL 21 Online Transfer 07 JUL 21 36,000.00 907,100.76
51451194
07 JUL 21 Account Transfer 07 JUL 21 36,775.00 870,325.76
51451192
07 JUL 21 Online Transfer 07 JUL 2

In [ ]:
prompt = """
You are a highly intelligent bot which specialises in data extraction,
data cleaning and data analysis of bank credit statements. Your task
is to extract transactions with their details from pdf statements from
bank account statements. The transactions have to be represented in CSV Format
 with the columns "Transaction Date", "Description",
"Debit/Withdrawal", "Credit/Deposit", "Balance".
The final response will be in the JSON format with the keys :
keys "user_bank", "opening_balance_date", "opening_balance",
"closing_balance", "transactions"

### Format ###
1. "Transaction Date" : to be output in the format "DD-MM-YYYY"

2. "Description": can contain multiple lines, every description contains
numerical value in one of the columns "Debit/Withdrawal", "Credit/Deposit".

3. "Debit/Withdrawal": will be the third last numerical version. can have
the keyword "CR" or can be a negative value.

4. "Credit/Deposit": will be the second last numerical value. can have
the keyword "DR".

5. "Balance": will be the last numerical value. there may be one value
for all the transactions for one day.

### Problem ###
If the Debit column and the credit column, do not have values in the same row
you will have to determine wether the value is Credit or Debit and add it in the
correct column.

### Solution ###
Use the formula:
If Balance of this row minus the balance of previous row is positive, the value is credit
else the value is debit

Return only a JSON response without anu note or prompt with the keys "user_bank", "opening_balance_date", "opening_balance",
"closing_balance", "transactions" for all the transactions in the following text.
Return values as Floating points with no special characters, only numericals values and decimal
point where possible.
###
{text}
###
"""

In [ ]:
allied_rsps = []
for pg in pages:
  response = get_completion(prompt.format(text=pg))
  allied_rsps.append(response)
[print(p + "\n") for p in allied_rsps]

{
"user_bank": "ABC (Private) Limited PK0010529",
"opening_balance_date": "01-07-2021",
"opening_balance": 914280.76,
"closing_balance": 747282.76,
"transactions": [
{
"Transaction Date": "05-07-2021",
"Description": "Cash Withdrawal",
"Debit/Withdrawal": 26610.00,
"Credit/Deposit": 0.00,
"Balance": 887670.76
},
{
"Transaction Date": "05-07-2021",
"Description": "Account Transfer",
"Debit/Withdrawal": 19100.00,
"Credit/Deposit": 0.00,
"Balance": 868570.76
},
{
"Transaction Date": "06-07-2021",
"Description": "Online Cash Deposit CR",
"Debit/Withdrawal": 0.00,
"Credit/Deposit": 25000.00,
"Balance": 893570.76
},
{
"Transaction Date": "06-07-2021",
"Description": "Transfer",
"Debit/Withdrawal": 0.00,
"Credit/Deposit": 130480.00,
"Balance": 1024050.76
},
{
"Transaction Date": "06-07-2021",
"Description": "Cash Withdrawal",
"Debit/Withdrawal": 80950.00,
"Credit/Deposit": 0.00,
"Balance": 943100.76
},
{
"Transaction Date": "07-07-2021",
"Description": "Online Transfer",
"Debit/Withdrawal": 3

[None, None, None]

In [ ]:
[print(json.loads(p)) for p in allied_rsps]

{'user_bank': 'ABC (Private) Limited PK0010529', 'opening_balance_date': '01-07-2021', 'opening_balance': 914280.76, 'closing_balance': 747282.76, 'transactions': [{'Transaction Date': '05-07-2021', 'Description': 'Cash Withdrawal', 'Debit/Withdrawal': 26610.0, 'Credit/Deposit': 0.0, 'Balance': 887670.76}, {'Transaction Date': '05-07-2021', 'Description': 'Account Transfer', 'Debit/Withdrawal': 19100.0, 'Credit/Deposit': 0.0, 'Balance': 868570.76}, {'Transaction Date': '06-07-2021', 'Description': 'Online Cash Deposit CR', 'Debit/Withdrawal': 0.0, 'Credit/Deposit': 25000.0, 'Balance': 893570.76}, {'Transaction Date': '06-07-2021', 'Description': 'Transfer', 'Debit/Withdrawal': 0.0, 'Credit/Deposit': 130480.0, 'Balance': 1024050.76}, {'Transaction Date': '06-07-2021', 'Description': 'Cash Withdrawal', 'Debit/Withdrawal': 80950.0, 'Credit/Deposit': 0.0, 'Balance': 943100.76}, {'Transaction Date': '07-07-2021', 'Description': 'Online Transfer', 'Debit/Withdrawal': 36000.0, 'Credit/Deposit

[None, None, None]

In [ ]:
len(allied_rsps)

3

In [ ]:
def validate(pages):
  prev_balance = float(pages[0]['opening_balance'])
  dfs = []
  for page in pages:
    if "transactions" in page:
      df = pd.DataFrame(page['transactions'])
      df.fillna(0.0, inplace=True)
      for i, row in df.iterrows():
        db, cr, b = row[2:5]
        if float(b) != round(prev_balance + float(cr) - float(db),2):
          if float(b) != round(prev_balance - float(cr) + float(db),2)  :

            print(f"index {i} | {float(b)} != {prev_balance} + {float(cr)} -{ float(db)} = { round(prev_balance + float(cr) - float(db),2)}")
            print(f"{float(b)} != {prev_balance} - {float(cr)} +{ float(db)} = { round(prev_balance - float(cr) + float(db),2)}")
            print(f"{row}\n\n")
            # raise BaseException(f"Inconsistency error: Missing Transactions between previous row: {df.iloc[i-1]} \nANd\n {row}")
            print(f"Inconsistency error: Missing Transactions between previous row: \n{df.iloc[i-1]} \nANd\n {row}\n\n\n")
          else:
            df.iloc[i,3], df.iloc[i,2] = df.iloc[i,2].copy(), df.iloc[i,3].copy()
        prev_balance = float(b)
      dfs.append(df)
  concatenated_df = pd.concat(dfs)
  concatenated_df.reset_index(drop=True, inplace=True)
  return concatenated_df


In [ ]:
res = validate([json.loads(p) for p in allied_rsps])
res.to_csv("allied_csv.csv")

index 0 | 2018608.76 != 1507258.76 + 0.0 -2102070.0 = -594811.24
2018608.76 != 1507258.76 - 0.0 +2102070.0 = 3609328.76
Transaction Date         21-06-2022
Description         Online Transfer
Debit/Withdrawal          2102070.0
Credit/Deposit                  0.0
Balance                  2018608.76
Name: 0, dtype: object


Inconsistency error: Missing Transactions between previous row: 
Transaction Date      30-06-2022
Description         Cash Deposit
Debit/Withdrawal             0.0
Credit/Deposit            1200.0
Balance                591258.76
Name: 23, dtype: object 
ANd
 Transaction Date         21-06-2022
Description         Online Transfer
Debit/Withdrawal          2102070.0
Credit/Deposit                  0.0
Balance                  2018608.76
Name: 0, dtype: object





In [ ]:
files = glob.glob("./pdfs/*.pdf")
for fil in files:
  f = fil.split('/')[-1][:-4]
  print(f"############ {f} ###########")
  try:
    text, pages = read_pdf(fil)
    with open(f'txts/{f}.txt',"w")as x:
      x.write(text)
    rsps = [json.loads(get_completion(prompt.format(text=pg))) for pg in pages]
    with open(f'jsons/{f}.json',"w")as x:
      json.dump(rsps, x, indent=4)
    res = validate(rsps)
    res.to_csv(f'csvs/{f}.json')
  except Exception as e:
    print(f"Error in {f} | {e}")


############ Meezan Bank (Premium Banking) ###########
index 0 | 99309.64 != 91623.91 + 7540.27 -0.0 = 99164.18
99309.64 != 91623.91 - 7540.27 +0.0 = 84083.64
Transaction Date                        04-07-2022
Description         Profit Payable On TDR- Credits
Debit/Withdrawal                               0.0
Credit/Deposit                             7540.27
Balance                                   99309.64
Name: 0, dtype: object


Inconsistency error: Missing Transactions between previous row: 
Transaction Date               03-09-2022
Description         Withholding Tax Debit
Debit/Withdrawal                  1144.92
Credit/Deposit                        0.0
Balance                         257321.32
Name: 15, dtype: object 
ANd
 Transaction Date                        04-07-2022
Description         Profit Payable On TDR- Credits
Debit/Withdrawal                               0.0
Credit/Deposit                             7540.27
Balance                                   99309.64
N

In [2]:
import tiktoken

def getTokens(tok, model='gpt-3.5-turbo'):
    encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')
    return encoding.encode(tok)

In [3]:
from prompts import prompt_ocr
with open('txts/1. Khushhali Bank 1 _KMBL_PKR.txt','r') as f:
    t = f.read()
context = prompt_ocr.format(text=t)
tokens = getTokens(context)
len(tokens)

5013